## Python set-up

In [1]:
# system imports
from math import isfinite
from functools import cache

# analytic imports
import matplotlib.pyplot as plt
import pandas as pd
import readabs as ra
from IPython.display import display
import statsmodels.api as sm  # type: ignore
from mgplot import (
    line_plot_finalise,
    finalise_plot,
    set_chart_dir,
    clear_chart_dir
)


In [2]:
# local imports
from abs_helper import QUARTERLY_CPI_RANGE
from henderson import hma

In [3]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# save charts in this notebook
plt.style.use("fivethirtyeight")
CHART_DIR = "./CHARTS/Inflation-Model/"
set_chart_dir(CHART_DIR)
clear_chart_dir()

# display charts in this notebook
SHOW = False

## Data capture

 ### Identify the data we want to use

In [4]:
@cache
def get_data() -> tuple[dict[str, pd.Series], pd.DataFrame]:
    """Get a dictionary of data items from the ABS."""

    wanted = {
        # "Series ID": ["Category ID", "single-excel-only table name", "Short Series Title"],
        "A3604510W": ["6401.0", "640106", "CPI_TM"],  # change from previous quarter
        "A2314867K": ["6427.0", "642701", "PPI"],  # change from previous year
        "A83895396W": ["6345.0", "634501", "WPI"],  # change from previous year
        "A84423050A": ["6202.0", "6202001", "UER"],  # percent of labour force
    }

    data, meta = {}, {}
    for series_id, (category_id, seo, title) in wanted.items():
        d, m = ra.read_abs_series(category_id, series_id, single_excel_only=seo)
        data[title] = d[series_id]
        meta[title] = m.loc[series_id]
    return data, pd.DataFrame(meta).T


_, META = get_data()
# check we have the correct variables
# META

### Collect and marshall that data
Assume we are only doind a one period prediction

In [5]:
def collect_data(subs: dict | None = None) -> tuple[pd.DataFrame, pd.Index, pd.Series]:
    """Collect the data for the inflation model.
    Arguments:
        subs: Dictionary of substitutions for forward values
    Returns:
        data: DataFrame containing the data
        exogenous: Index of exogenous variable names
        endogenous: Series of endogenous variables (ie CPI_TM)"""

    data_dict, _meta = get_data()
    data_dict["UER"] = ra.monthly_to_qtly(data_dict["UER"])
    data = pd.concat(data_dict, axis=1)

    # forward for one period prediction
    last = data.loc[data.index[-1], "CPI_TM"]
    if isfinite(last):
        next_period = data.index[-1] + 1
        new_index = data.index.append(pd.PeriodIndex([next_period]))
        data = data.reindex(new_index)

    # populate forward exogenous guesses into the model
    last = data.index[-1]
    if subs is not None:
        for k, v in subs.items():
            if subs == "CPI_TM" or isfinite(data.loc[last, k]):
                continue
            data.loc[last, k] = v

    # Adjust missing values in PPI and WPI because they are published after CPI
    # so we may need rolling average approximations to plug in
    for s in ("PPI", "WPI"):
        _ = "Do noting if next lines commented out ..."
        data[f"{s}_ADJ"] = data[s].where(
            data[s].notna(), other=data[s].rolling(4).mean().ffill()
        )
        # data = data.drop(columns=s)

    # smooth WPI/PPI
    h = 7
    for s in ("WPI", "PPI"):
        _ = "Do noting if next lines commented ..."
        data[f"{s}_HMA[{h}]"] = hma(data[f"{s}_ADJ"].dropna(), h)

    # provide differences
    for col in ("PPI_ADJ", "WPI_ADJ", "UER"):
        data[f"Δ{col}"] = data[f"{s}_ADJ"].diff(1)

    # add autoregression
    for ar in (1, 2):
        data[f"CPI_TM-{ar}"] = data["CPI_TM"].shift(ar)

    # add the constant
    data["const"] = 1.0

    # Add a COVID dummy
    data["covid"] = 0
    mask = (data.index >= "2020Q1") & (data.index <= "2021Q1")
    data.loc[mask, "covid"] = 1

    # remove early nans
    remember = data.iloc[-1]
    data = pd.concat([data.dropna(), remember.to_frame().T])

    # exogenous and out-of-sample endogenous where known variables
    exogenous = data.columns.difference(["CPI_TM"])
    endogenous = data.index[-1]
    return data, exogenous, endogenous


DATA, EXOG, ENDOG = collect_data(subs={"UER": 4.1, "WPI": 4.0, "PPI": 4.3})
# DATA.tail()

In [6]:
EXOG

Index(['CPI_TM-1', 'CPI_TM-2', 'PPI', 'PPI_ADJ', 'PPI_HMA[7]', 'UER', 'WPI',
       'WPI_ADJ', 'WPI_HMA[7]', 'const', 'covid', 'ΔPPI_ADJ', 'ΔUER',
       'ΔWPI_ADJ'],
      dtype='object')

In [7]:
ENDOG

Period('2025Q3', 'Q-DEC')

## Build a simple multiple regression model

In [8]:
forecasts = {}


def run_model(data, exog: list[str], title: str, endog: pd.Series | None):
    """Run the model for the given data and exogenous variables."""

    # Fit the model
    y = data["CPI_TM"].dropna()
    X = data.loc[y.index, exog]
    Xnew = data.loc[data.index.difference(X.index), exog]
    print(X.tail())
    model = sm.OLS(y, X)
    fit = model.fit()
    print(fit.summary())

    # QQ plot
    fig = sm.qqplot(fit.resid, line="s")
    name = title.split(" vs ")[1].rsplit(" ", 1)[0]
    finalise_plot(
        fig.axes[0],
        title=f"QQ Plot: {name}",
        xlabel="Theoretical Quantiles",
        ylabel="Sample Quantiles",
        show=SHOW,
    )

    # can we forecast?
    can_forecast = endog is not None and len(Xnew) > 0
    if can_forecast:
        print("Forecasting using:")
        display(Xnew)
        ynewpred = fit.predict(Xnew)  # predict out of sample
        display(ynewpred)
        forecasts[title] = ynewpred.iloc[0]
        ynewpred[y.index[-1]] = y.iloc[-1]  # start from actual
        quarterly = ynewpred.iloc[1]
        annual = (
            (((pd.concat([y.iloc[-3:-1], ynewpred]) / 100) + 1).cumprod() - 1) * 100
        ).iloc[-1]
        projection = (
            f"Projection {ynewpred.index[-1]}: A:{annual:.2f}% Q:{quarterly:.2f}%"
        )
    projection = projection if can_forecast else ""

    # plot avtual vs predicted
    mlist = [f"({x}*{round(y, 3)})" for x, y in fit.params.items()]
    m = "ypred = " + (" + ").join(mlist)
    print("Model: ", m)
    ypred = fit.predict(X)
    model_frame = pd.DataFrame({"Actual": y, "Within sample predicted": ypred})
    if can_forecast:
        model_frame = model_frame.reindex(model_frame.index.union(ynewpred.index))
        model_frame["Out of sample forecast"] = ynewpred
    line_plot_finalise(
        model_frame,
        title=title,
        ylabel="Inflation % per quarter",
        color=("cornflowerblue", "darkorange", "darkred"),
        width=(1.5, 2, 3),
        axhspan=QUARTERLY_CPI_RANGE,
        lfooter=m if len(m) < 80 else "",
        y0=True,
        legend=True,
        show=SHOW,
    )

In [9]:
# Simple everything model
title_ = "Trimmed Mean Inflation vs All Exogenous Model"
run_model(DATA, EXOG.to_list(), title_, ENDOG)

        CPI_TM-1  CPI_TM-2  PPI  PPI_ADJ  PPI_HMA[7]       UER  WPI  WPI_ADJ  \
2024Q1       0.9       1.2  4.3      4.3    4.393986  3.902216  4.0      4.0   
2024Q2       1.0       0.9  4.8      4.8    4.406434  4.050091  4.1      4.1   
2024Q3       0.8       1.0  3.9      3.9    4.098252  4.126066  3.6      3.6   
2024Q4       0.8       0.8  3.7      3.7    3.771958  4.001787  3.2      3.2   
2025Q1       0.5       0.8  3.7      3.7    3.767995  4.078707  3.4      3.4   

        WPI_HMA[7]  const  covid  ΔPPI_ADJ  ΔUER  ΔWPI_ADJ  
2024Q1    4.158601    1.0    0.0       0.2   0.2       0.2  
2024Q2    3.929650    1.0    0.0       0.5   0.5       0.5  
2024Q3    3.601469    1.0    0.0      -0.9  -0.9      -0.9  
2024Q4    3.357133    1.0    0.0      -0.2  -0.2      -0.2  
2025Q1    3.386199    1.0    0.0       0.0   0.0       0.0  
                            OLS Regression Results                            
Dep. Variable:                 CPI_TM   R-squared:                       0

,CPI_TM-1,CPI_TM-2,PPI,PPI_ADJ,PPI_HMA[7],UER,WPI,WPI_ADJ,WPI_HMA[7],const,covid,ΔPPI_ADJ,ΔUER,ΔWPI_ADJ
2025Q3,0.6,0.7,4.3,4.3,4.234988,4.1,4.0,4.0,3.903253,1.0,0.0,0.275,0.275,0.275


2025Q3    0.776439
Freq: Q-DEC, dtype: float64

Model:  ypred = (CPI_TM-1*0.408) + (CPI_TM-2*0.226) + (PPI*-0.048) + (PPI_ADJ*-0.048) + (PPI_HMA[7]*0.129) + (UER*-0.038) + (WPI*0.081) + (WPI_ADJ*0.081) + (WPI_HMA[7]*-0.166) + (const*0.383) + (covid*-0.029) + (ΔPPI_ADJ*0.017) + (ΔUER*0.017) + (ΔWPI_ADJ*0.017)


In [10]:
# REMOVE some cooefficients that are not significantly different from zero
exog_1 = EXOG.difference(
    [
        "WPI",
        "WPI_ADJ",
        "WPI_HMA[7]",
        "PPI",
        "PPI_ADJ",
        "UER",
        "ΔPPI_ADJ",
        "ΔUER",
        "ΔWPI_ADJ",
    ]
).to_list()
title_ = "Trimmed Mean Inflation vs AR2 Model Predicted"
run_model(DATA, exog_1, title_, ENDOG)

        CPI_TM-1  CPI_TM-2  PPI_HMA[7]  const  covid
2024Q1       0.9       1.2    4.393986    1.0    0.0
2024Q2       1.0       0.9    4.406434    1.0    0.0
2024Q3       0.8       1.0    4.098252    1.0    0.0
2024Q4       0.8       0.8    3.771958    1.0    0.0
2025Q1       0.5       0.8    3.767995    1.0    0.0
                            OLS Regression Results                            
Dep. Variable:                 CPI_TM   R-squared:                       0.622
Model:                            OLS   Adj. R-squared:                  0.607
Method:                 Least Squares   F-statistic:                     39.94
Date:                Wed, 30 Jul 2025   Prob (F-statistic):           9.74e-20
Time:                        13:52:31   Log-Likelihood:                 26.402
No. Observations:                 102   AIC:                            -42.80
Df Residuals:                      97   BIC:                            -29.68
Df Model:                           4             

,CPI_TM-1,CPI_TM-2,PPI_HMA[7],const,covid
2025Q3,0.6,0.7,4.234988,1.0,0.0


2025Q3    0.736007
Freq: Q-DEC, dtype: float64

Model:  ypred = (CPI_TM-1*0.437) + (CPI_TM-2*0.184) + (PPI_HMA[7]*0.049) + (const*0.138) + (covid*-0.041)


In [11]:
exog_ = ["CPI_TM-1", "UER", "covid", "const"]
title_ = "Trimmed Mean Inflation vs AR1 Model Predicted"
run_model(DATA, exog_, title_, ENDOG)

        CPI_TM-1       UER  covid  const
2024Q1       0.9  3.902216    0.0    1.0
2024Q2       1.0  4.050091    0.0    1.0
2024Q3       0.8  4.126066    0.0    1.0
2024Q4       0.8  4.001787    0.0    1.0
2025Q1       0.5  4.078707    0.0    1.0
                            OLS Regression Results                            
Dep. Variable:                 CPI_TM   R-squared:                       0.597
Model:                            OLS   Adj. R-squared:                  0.585
Method:                 Least Squares   F-statistic:                     48.38
Date:                Wed, 30 Jul 2025   Prob (F-statistic):           2.85e-19
Time:                        13:52:32   Log-Likelihood:                 23.097
No. Observations:                 102   AIC:                            -38.19
Df Residuals:                      98   BIC:                            -27.69
Df Model:                           3                                         
Covariance Type:            nonrobust      

,CPI_TM-1,UER,covid,const
2025Q3,0.6,4.1,0.0,1.0


2025Q3    0.72942
Freq: Q-DEC, dtype: float64

Model:  ypred = (CPI_TM-1*0.608) + (UER*-0.078) + (covid*-0.083) + (const*0.683)


In [12]:
# Multi-model
# exog_ = ['CPI_TM-1', 'CPI_TM-2', "UER", "WPI_HMA[7]", "PPI_HMA[7]",  "const"]
# exog_ = ['CPI_TM-1', 'CPI_TM-2', "PPI_HMA[7]"]  # <-- the best model (no employment)
# exog_ = ['CPI_TM-1', 'CPI_TM-2', "WPI_HMA[7]"]
exog_ = ["CPI_TM-1", "CPI_TM-2", "ΔUER", "PPI_HMA[7]", "WPI_HMA[7]", "covid"]
title_ = "Trimmed Mean Inflation vs Multi Model Predicted"
run_model(DATA, exog_, title_, ENDOG)

        CPI_TM-1  CPI_TM-2  ΔUER  PPI_HMA[7]  WPI_HMA[7]  covid
2024Q1       0.9       1.2   0.2    4.393986    4.158601    0.0
2024Q2       1.0       0.9   0.5    4.406434    3.929650    0.0
2024Q3       0.8       1.0  -0.9    4.098252    3.601469    0.0
2024Q4       0.8       0.8  -0.2    3.771958    3.357133    0.0
2025Q1       0.5       0.8   0.0    3.767995    3.386199    0.0
                                 OLS Regression Results                                
Dep. Variable:                 CPI_TM   R-squared (uncentered):                   0.937
Model:                            OLS   Adj. R-squared (uncentered):              0.933
Method:                 Least Squares   F-statistic:                              238.8
Date:                Wed, 30 Jul 2025   Prob (F-statistic):                    2.16e-55
Time:                        13:52:32   Log-Likelihood:                          26.430
No. Observations:                 102   AIC:                                     -40.86


,CPI_TM-1,CPI_TM-2,ΔUER,PPI_HMA[7],WPI_HMA[7],covid
2025Q3,0.6,0.7,0.275,4.234988,3.903253,0.0


2025Q3    0.743302
Freq: Q-DEC, dtype: float64

Model:  ypred = (CPI_TM-1*0.455) + (CPI_TM-2*0.23) + (ΔUER*0.044) + (PPI_HMA[7]*0.037) + (WPI_HMA[7]*0.036) + (covid*0.03)


In [13]:
# mean of all models
pd.Series(forecasts).mean()

np.float64(0.7462919006658991)

## Backtest

In [14]:
BACK_DATA = DATA.iloc[:-1]
BACK_INDEX = BACK_DATA.index[-1]
ACTUAL = BACK_DATA.iloc[-1, 0]
BACK_DATA.iloc[-1, 0] = None
BACK_DATA.tail(), BACK_INDEX

(        CPI_TM  PPI  WPI       UER  PPI_ADJ  WPI_ADJ  WPI_HMA[7]  PPI_HMA[7]  \
 2024Q1     1.0  4.3  4.0  3.902216      4.3      4.0    4.158601    4.393986   
 2024Q2     0.8  4.8  4.1  4.050091      4.8      4.1    3.929650    4.406434   
 2024Q3     0.8  3.9  3.6  4.126066      3.9      3.6    3.601469    4.098252   
 2024Q4     0.5  3.7  3.2  4.001787      3.7      3.2    3.357133    3.771958   
 2025Q1     NaN  3.7  3.4  4.078707      3.7      3.4    3.386199    3.767995   
 
         ΔPPI_ADJ  ΔWPI_ADJ  ΔUER  CPI_TM-1  CPI_TM-2  const  covid  
 2024Q1       0.2       0.2   0.2       0.9       1.2    1.0    0.0  
 2024Q2       0.5       0.5   0.5       1.0       0.9    1.0    0.0  
 2024Q3      -0.9      -0.9  -0.9       0.8       1.0    1.0    0.0  
 2024Q4      -0.2      -0.2  -0.2       0.8       0.8    1.0    0.0  
 2025Q1       0.0       0.0   0.0       0.5       0.8    1.0    0.0  ,
 Period('2025Q1', 'Q-DEC'))

In [15]:
exog_ = ["CPI_TM-1", "CPI_TM-2", "ΔUER", "PPI_HMA[7]", "WPI_HMA[7]", "covid"]
title_ = "BT: Trimmed Mean Inflation vs Multi Model Predicted"
run_model(BACK_DATA, exog_, title_, ENDOG)

        CPI_TM-1  CPI_TM-2  ΔUER  PPI_HMA[7]  WPI_HMA[7]  covid
2023Q4       1.2       0.9   0.3    4.064755    4.153147    0.0
2024Q1       0.9       1.2   0.2    4.393986    4.158601    0.0
2024Q2       1.0       0.9   0.5    4.406434    3.929650    0.0
2024Q3       0.8       1.0  -0.9    4.098252    3.601469    0.0
2024Q4       0.8       0.8  -0.2    3.771958    3.357133    0.0
                                 OLS Regression Results                                
Dep. Variable:                 CPI_TM   R-squared (uncentered):                   0.937
Model:                            OLS   Adj. R-squared (uncentered):              0.933
Method:                 Least Squares   F-statistic:                              234.2
Date:                Wed, 30 Jul 2025   Prob (F-statistic):                    1.26e-54
Time:                        13:52:32   Log-Likelihood:                          25.685
No. Observations:                 101   AIC:                                     -39.37


,CPI_TM-1,CPI_TM-2,ΔUER,PPI_HMA[7],WPI_HMA[7],covid
2025Q1,0.5,0.8,0.0,3.767995,3.386199,0.0


2025Q1    0.671513
Freq: Q-DEC, dtype: float64

Model:  ypred = (CPI_TM-1*0.458) + (CPI_TM-2*0.229) + (ΔUER*0.044) + (PPI_HMA[7]*0.036) + (WPI_HMA[7]*0.036) + (covid*0.03)


## Finished

In [16]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-07-30 13:52:32

Python implementation: CPython
Python version       : 3.13.5
IPython version      : 9.4.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

matplotlib : 3.10.3
readabs    : 0.1.4
mgplot     : 0.2.12
IPython    : 9.4.0
pandas     : 2.3.1
statsmodels: 0.14.5

Watermark: 2.5.0



In [17]:
print("Finished")

Finished
